# Finding locations in Kauai for a State Farm Dealership
## Final Report for Coursera IBM Applied Data Science Capstone
## *Charles Vuono*

### Introduction / Business Problem
State Farm Insurance is a national chain of independently owned insurance dealerships who contract with State Farm for the right to establish a dealership which will exclusively sell State Farm insurance products from a retail location. Currently, the island of Kauai, Hawaii has no State Farm dealerships. Supposing that we are interested in opening a State Farm dealership on Kauai, the purpsose of this investigation is to understand the characteristics of locations that currently support State Farm Dealerships (elsewhere in Hawaii or on the US Mainland) and compare them with various areas of Kauai to determine suitable candidates to establish a State Farm dealership. We will study existing State Farm Dealership locations through a variety of descriptive and predictive analytic techniques on neighborhood data, and use these results to find similar Kauai neighborhoods.

### Data
We will use two primary data sources to conduct our investigation:
1. Location data on every existing State Farm dealership in the United States, including longitude, latitude and full address (approximately 19,000 records)
2. Neighborhood data from the FourSquare Developer API regarding the types of commercial establishments in proximity to these dealerships.

We shall now load the State Farm data:



In [14]:
# The code was removed by Watson Studio for sharing.

The State Farm data consists of 18876 rows and 18 columns.
A sample of the data is as follows:


,latitude,longitude,address1,address2,Annuities,Auto Insurance,Banking Products,Business Insurance,city,Health Insurance,Home Insurance,Home Lending,Life Insurance,Mutual Funds,phone,Property Insurance,state,zip
0,42.885943,-77.280766,7 Coy St,NaN,1.0,1.0,1.0,1.0,Canandaigua,1.0,1.0,NaN,1.0,1.0,5853946840,1.0,NY,14424-1506
1,45.863906,-95.373885,2633 Jefferson St Suite 803b,NaN,1.0,1.0,1.0,1.0,Alexandria,1.0,1.0,NaN,1.0,NaN,3207622394,1.0,MN,56308
2,41.051858,-76.220358,504 3rd Street,PO Box 502,1.0,1.0,1.0,1.0,Nescopeck,1.0,1.0,NaN,1.0,NaN,5707521191,1.0,PA,18635-0502
3,41.140139,-112.037307,928 W 1800 N Suite H,NaN,1.0,1.0,NaN,1.0,Clinton,1.0,1.0,NaN,1.0,NaN,8017792000,1.0,UT,84015-6608
4,38.475771,-81.079581,2279 Main Street,NaN,1.0,1.0,1.0,1.0,Clay,1.0,1.0,NaN,1.0,NaN,3045872899,1.0,WV,25043-7001
